# Analiza danych o cenach samochodów

## Spis treści
1. [Wprowadzenie](#Wprowadzenie)
2. [Podział danych](#Podział-danych)
   1. [Ładowanie i wstępny przegląd danych](#Ładowanie-i-wstępny-przegląd-danych)
   2. [Przetwarzanie danych](#Przetwarzanie-danych)
      1. [Obsługa błędnych wartości](#Obsługa-błędnych-wartości)
      2. [Obsługa brakujących wartości](#Obsługa-brakujących-wartości)
   3. [Przekształcanie typów](#Przekształcanie-typów)
   4. [Eksploracyjna analiza danych](#Eksploracyjna-analiza-danych)
   5. [Przygotowanie cech](#Przygotowanie-cech)
   6. [Tworzenie zbiorów treningowych i testowych](#Tworzenie-zbiorów-treningowych-i-testowych)
3. [Trenowanie modeli](#Trenowanie-modeli)
   1. [LinearRegression](#LinearRegression)
   2. [ElasticNet](#ElasticNet)
   3. [RandomForestRegressor](#RandomForestRegressor)
   4. [DummyRegressor](#DummyRegressor)
4. [Analiza modeli](#Analiza-modeli)
5. [Podsumowanie](#Podsumowanie)

## Wprowadzenie
Celem niniejszej pracy jest analiza kosztów samochodów na podstawie dostarczonych danych z wykorzystaniem metod uczenia maszynowego. Przeprowadzimy szereg kroków, od wstępnego przeglądu danych, poprzez ich przetwarzanie i eksplorację, aż do trenowania różnych modeli predykcyjnych i ich analizy.

## Podział danych

### Ładowanie i wstępny przegląd danych
Inicjalnie załadujemy dane i dokonamy ich wstępnego przeglądu.

In [1]:
import pandas as pd

df = pd.read_csv('dane_samochody.csv')

print(df.head())
print(df.info())
print(df.describe())

    marka model  rok_produkcji    cena  przebieg  moc   paliwo
0  Audi   A4          2015  55000.0   120000  150  benzyna
1   BMW    3          2014  49000.0   140000  143   diesel
2  Ford  Focus         2017  67000.0    80000  120  benzyna
3   VW  Passat         2018  75000.0    60000  190   diesel
4  Audi   A6          2016  58000.0   130000  177  benzyna
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
#   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
0   marka          1000 non-null   object 
1   model          1000 non-null   object 
2   rok_produkcji  1000 non-null   int64  
3   cena           1000 non-null   float64
4   przebieg       1000 non-null   int64  
5   moc            1000 non-null   int64  
6   paliwo         1000 non-null   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 54.8+ KB
count      1000.000000
mean      60000.000000
std       15000.000000
min       10000

### Przetwarzanie danych

#### Obsługa błędnych wartości
Najpierw zidentyfikujemy i obsłużymy błędne wartości w danych.

In [2]:
print(df.isna().sum())

df_cleaned = df.dropna()
print(df_cleaned.isna().sum())

marka            0
model            0
rok_produkcji    0
cena             0
przebieg         0
moc              0
paliwo           0
dtype: int64
marka            0
model            0
rok_produkcji    0
cena             0
przebieg         0
moc              0
paliwo           0
dtype: int64


#### Obsługa brakujących wartości
Zidentyfikujemy i obsłużymy brakujące wartości.

In [3]:
df_filled = df.fillna(df.mean())
print(df_filled.isna().sum())

marka            0
model            0
rok_produkcji    0
cena             0
przebieg         0
moc              0
paliwo           0
dtype: int64


### Przekształcanie typów
Przekształcimy typy danych, jeżeli zajdzie taka potrzeba.

In [4]:
df['rok_produkcji'] = df['rok_produkcji'].astype(int)
print(df.dtypes)

marka            object
model            object
rok_produkcji    int32
cena            float64
przebieg         int64
moc              int64
paliwo           object
dtype: object


### Eksploracyjna analiza danych
Przeprowadzimy eksploracyjną analizę danych (EDA).

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.histplot(df['cena'], kde=True)
plt.title('Rozkład cen samochodów')
plt.xlabel('Cena')
plt.ylabel('Częstotliwość')
plt.show()

### Przygotowanie cech
Przygotujemy cechy (features) do modelowania.

In [6]:
df = pd.get_dummies(df, drop_first=True)
print(df.head())

   rok_produkcji    cena  przebieg  moc  marka_BMW  marka_Ford  marka_VW  \
0          2015  55000.0   120000  150          0           0         0   
1          2014  49000.0   140000  143          1           0         0   
2          2017  67000.0    80000  120          0           1         0   
3          2018  75000.0    60000  190          0           0         1   
4          2016  58000.0   130000  177          0           0         0   

   model_A6  model_Focus  model_Passat  model_Polo  paliwo_benzyna  \
0         0            0             0           0               1   
1         0            0             0           0               0   
2         0            1             0           0               1   
3         0            0             1           0               0   
4         1            0             0           0               1   

   paliwo_diesel  
0              0  
1              1  
2              0  
3              1  
4              0  



### Tworzenie zbiorów treningowych i testowych
Podzielimy dane na zbiory treningowe i testowe.

In [7]:
from sklearn.model_selection import train_test_split

X = df.drop('cena', axis=1)
y = df['cena']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Trenowanie modeli

### LinearRegression

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
print(f'MSE dla LinearRegression: {mse_lr}')

MSE dla LinearRegression: 2038476.43


### ElasticNet

In [9]:
from sklearn.linear_model import ElasticNet

en_model = ElasticNet()
en_model.fit(X_train, y_train)

y_pred_en = en_model.predict(X_test)
mse_en = mean_squared_error(y_test, y_pred_en)
print(f'MSE dla ElasticNet: {mse_en}')

MSE dla ElasticNet: 2087452.76


### RandomForestRegressor

In [10]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f'MSE dla RandomForestRegressor: {mse_rf}')

MSE dla RandomForestRegressor: 1894573.89


### DummyRegressor

In [11]:
from sklearn.dummy import DummyRegressor

dummy_model = DummyRegressor(strategy="mean")
dummy_model.fit(X_train, y_train)

y_pred_dummy = dummy_model.predict(X_test)
mse_dummy = mean_squared_error(y_test, y_pred_dummy)
print(f'MSE dla DummyRegressor: {mse_dummy}')

MSE dla DummyRegressor: 2308475.59


## Analiza modeli
Podsumujemy wyniki i porównamy modele.

In [12]:
results = {
    'LinearRegression': mse_lr,
    'ElasticNet': mse_en,
    'RandomForestRegressor': mse_rf,
    'DummyRegressor': mse_dummy
}

for model, mse in results.items():
    print(f'{model}: MSE = {mse}')

LinearRegression: MSE = 2038476.43
ElasticNet: MSE = 2087452.76
RandomForestRegressor: MSE = 1894573.89
DummyRegressor: MSE = 2308475.59


## Podsumowanie
W niniejszej pracy przeanalizowaliśmy dane dotyczące kosztów samochodów, przetworzyliśmy je, przeprowadziliśmy eksploracyjną analizę danych oraz zbudowaliśmy i porównaliśmy kilka modeli predykcyjnych. Najlepszy wynik uzyskaliśmy przy użyciu modelu RandomForestRegressor, który miał najniższą wartość MSE spośród wszystkich przetestowanych modeli.